<h1><center>Простые векторные модели текста</center></h1>

<img src="pipeline_vec.png" alt="pipeline.png" style="width: 400px;"/>

### Задача: классификация твитов по тональности

В этом занятии мы познакомимся с распространенной задачей в анализе текстов: с классификацией текстов на классы.

В рассмотренном тут примере классов будет два: положительный и отрицательный, такую постановку этой задачи обычно называют классификацией по тональности или sentiment analysis.

Классификацию по тональности используют, например, в рекомендательных системах и при анализе отзывов клиентов, чтобы понять, понравилось ли людям кафе, кино, etc.

Более подробно мы рассмотрим данную задачу и познакомимся с более сложными методами её решения в семинаре 3, а здесь разберем простые подходы, основанные на методе мешка слов.

У нас есть [данные постов в твиттере](http://study.mokoron.com/), про из которых каждый указано, как он эмоционально окрашен: положительно или отрицательно. 

**Задача**: построить модель, которая по тексту поста предсказывает его эмоциональную окраску.


Скачиваем данные: [положительные](https://drive.google.com/file/d/1mW_fUtYmRF19AXVySU0gJOIgx0-1EFgD/view?usp=sharing), [отрицательные](https://drive.google.com/file/d/1ZnsFuf-yfO3UEHlIpk7TTqfKkEMdm1EQ/view?usp=sharing).

In [1]:
# если у вас линукс / мак / collab или ещё какая-то среда, в которой работает wget, можно так:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mW_fUtYmRF19AXVySU0gJOIgx0-1EFgD' -O positive.csv
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ZnsFuf-yfO3UEHlIpk7TTqfKkEMdm1EQ' -O negative.csv

--2021-10-16 14:39:43--  https://docs.google.com/uc?export=download&id=1mW_fUtYmRF19AXVySU0gJOIgx0-1EFgD
Resolving docs.google.com (docs.google.com)... 74.125.195.102, 74.125.195.139, 74.125.195.113, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-94-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8f98nt1rspfi1p895kg1upeu1snmg81j/1634395125000/10227726563468148216/*/1mW_fUtYmRF19AXVySU0gJOIgx0-1EFgD?e=download [following]
--2021-10-16 14:39:44--  https://doc-04-94-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8f98nt1rspfi1p895kg1upeu1snmg81j/1634395125000/10227726563468148216/*/1mW_fUtYmRF19AXVySU0gJOIgx0-1EFgD?e=download
Resolving doc-04-94-docs.googleusercontent.com (doc-04-94-docs.googleusercontent.com)... 74.125.142.132, 2607:f8b0:400e:c08::84
Connecting to doc-04-94-docs.googleusercontent.com (doc-04-94-d

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [3]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [4]:
df.sample(5)

,text,label
7066,"Сеструха второй палец ломает. Бляя, опять:((",negative
49624,"Все очень нравятся, но нужно выбрать) http://t.co/wZs4zkTdVS http://t.co/OyNbBgID0r",positive
94008,@_ash_tan_ да и там не нужна! Жизнь-боль((((,negative
108320,"@assaron Типа того :) Они уже не в первый раз пытаются договориться, но там куча технических сложностей, помимо политических.",positive
90828,"Перебирала фото и нашла кое-что.\nВремя пускать слюни, сладкоежки Х) http://t.co/fVdhrWFRAU",positive


Воспользуемся функцией для предобработки текста, которую мы написали в прошлом семинаре:

In [6]:
!pip3 install pymorphy2

     |████████████████████████████████| 55 kB 2.5 MB/s 
     |████████████████████████████████| 8.2 MB 16.0 MB/s 


In [9]:
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

m = MorphAnalyzer()
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [11]:
from multiprocessing import Pool
from tqdm import tqdm

with Pool(4) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))
    
df['lemmas'] = lemmas
df.sample(5)

100%|██████████| 226834/226834 [07:31<00:00, 501.99it/s]


,text,label,lemmas
55503,@makeshka да. Причем относительно только самого себя ;),positive,makeshka причём относительно
10074,"@kristinadorozh1 несколько минут назад закончили разговаривать по скайпу,так что я уже в курсе)",positive,kristinadorozh несколько минута назад закончить разговаривать скайп курс
66602,"Что сегодня за день вообще..терракты,расстрелы,политспоры,кражи...еще и бадун вот(",negative,сегодня день вообще терракт расстрел политспорый кража бадун
68070,RT @porsheo_o: @Xaibull Я заметил Хуярит не по детский,negative,porsheo_o xaibull заметить хуярить детский
6271,xxx: у меня сексуальное / расстройство / yyy: Ты серьезно? / xxx: серьезней некуда( / xxx: никто не дает! / расстраиваюсь...,negative,сексуальный расстройство серьёзно серьёзный некуда никто давать расстраиваться


Разбиваем на train и test:

In [12]:
x_train, x_test, y_train, y_test = train_test_split(df.lemmas, df.label)

## Мешок слов (Bag of Words, BoW)


In [13]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer

... Но сперва пару слов об n-граммах. Что такое n-граммы:

In [14]:
from nltk import ngrams

In [15]:
sent = 'Если б мне платили каждый раз'.split()
list(ngrams(sent, 1)) # униграммы

[('Если',), ('б',), ('мне',), ('платили',), ('каждый',), ('раз',)]

In [16]:
list(ngrams(sent, 2)) # биграммы

[('Если', 'б'),
 ('б', 'мне'),
 ('мне', 'платили'),
 ('платили', 'каждый'),
 ('каждый', 'раз')]

In [17]:
list(ngrams(sent, 3)) # триграммы

[('Если', 'б', 'мне'),
 ('б', 'мне', 'платили'),
 ('мне', 'платили', 'каждый'),
 ('платили', 'каждый', 'раз')]

In [18]:
list(ngrams(sent, 5)) # ... пентаграммы?

[('Если', 'б', 'мне', 'платили', 'каждый'),
 ('б', 'мне', 'платили', 'каждый', 'раз')]

Итак, мы хотим преобразовать наши обработанные данные в вектора с помощью мешка слов. Мешок слов можно строить как для отдельных слов (лемм в нашем случае), так и для n-грамм, и это может улучшать качество. 

Объект `CountVectorizer` делает простую вещь:
* строит для каждого документа (каждой пришедшей ему строки) вектор размерности `n`, где `n` -- количество слов или n-грам во всём корпусе
* заполняет каждый i-тый элемент количеством вхождений слова в данный документ

In [19]:
vec = CountVectorizer(ngram_range=(1, 1)) # строим BoW для слов
bow = vec.fit_transform(x_train) 

ngram_range отвечает за то, какие n-граммы мы используем в качестве признаков:<br/>
ngram_range=(1, 1) -- униграммы<br/>
ngram_range=(3, 3) -- триграммы<br/>
ngram_range=(1, 3) -- униграммы, биграммы и триграммы.

В vec.vocabulary_ лежит словарь: соответствие слов и их индексов в словаре:

In [20]:
list(vec.vocabulary_.items())[:10]

[('знать', 113708),
 ('готовиться', 104951),
 ('задание', 111440),
 ('nika__av', 56469),
 ('самый', 148926),
 ('главное', 104168),
 ('остаться', 133946),
 ('дело', 106616),
 ('починить', 141719),
 ('телефон', 156299)]

In [21]:
bow[0]

<1x168764 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

Теперь у нас есть вектора, на которых можно обучать модели! 

In [22]:
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Посмотрим на качество классификации на тестовой выборке. Для этого выведем classification_report из модуля [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics)

В качестве целевой метрики качества будем рассматривать macro average f1-score.

In [23]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.74      0.73      0.74     28491
    positive       0.74      0.75      0.74     28218

    accuracy                           0.74     56709
   macro avg       0.74      0.74      0.74     56709
weighted avg       0.74      0.74      0.74     56709



Попробуем сделать то же самое для триграмм:

In [24]:
vec = CountVectorizer(ngram_range=(3, 3))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 300)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.97      0.53      0.69     51602
    positive       0.15      0.85      0.26      5107

    accuracy                           0.56     56709
   macro avg       0.56      0.69      0.47     56709
weighted avg       0.90      0.56      0.65     56709



Видим, что качество существенно хуже. Ниже мы поймем, почему это так.

## TF-IDF векторизация

`TfidfVectorizer` делает то же, что и `CountVectorizer`, но в качестве значений – tf-idf каждого слова.

Как считается tf-idf:

TF (term frequency) – относительная частотность слова в документе:
$$ TF(t,d) = \frac{n_t}{\sum_k n_k} $$

`t` -- слово (term), `d` -- документ, $n_t$ -- количество вхождений слова, $n_k$ -- количество вхождений остальных слов

IDF (inverse document frequency) – обратная частота документов, в которых есть это слово:
$$ IDF(t, D) = \mbox{log} \frac{|D|}{|{d : t \in d}|} $$

`t` -- слово (term), `D` -- коллекция документов

Перемножаем их:
$$TFIDF(t,d,D) = TF(t,d) \times IDF(i, D)$$

Ключевая идея этого подхода – если слово часто встречается в одном документе, но в целом по корпусу встречается в небольшом 
количестве документов, у него высокий TF-IDF.

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.70      0.75      0.73     26369
    positive       0.77      0.72      0.75     30340

    accuracy                           0.74     56709
   macro avg       0.74      0.74      0.74     56709
weighted avg       0.74      0.74      0.74     56709



В этот раз получилось хуже, чем с помощью простого CountVectorizer, то есть использование tf-idf не дало улучшений в качестве. 

## О важности эксплоративного анализа

Иногда в ходе стандартного препроцессинга теряются важные признаки. Посмотрим, что будет если не убирать пунктуацию?

In [27]:
df.sample()

,text,label,lemmas
16930,"Бляяя печаль ( я умудрился забыть наушники, походу снова буду по кольцу завтра катать",negative,бляять печаль умудриться забыть наушник поход снова кольцо завтра катать


In [28]:
df['new_lemmas'] = df.text.apply(lambda x: x.lower())
df.sample(3)

,text,label,lemmas,new_lemmas
87798,"@Nina_Leni_W у меня и так настроение не очень, в теперь и это:(",negative,nina_leni_w настроение очень,"@nina_leni_w у меня и так настроение не очень, в теперь и это:("
50214,@Ivp76Natacha76 меня ма не отпустит:( я уроки не сделала:(((,negative,natacha отпустить урок сделать,@ivp76natacha76 меня ма не отпустит:( я уроки не сделала:(((
89330,"@juliamayko @and_Possum Да графиня, по-моему, кончает регулярно. Только мы неудовлетворенными остаемся(((",negative,juliamayko and_possum графиня кончать регулярно неудовлетворённый оставаться,"@juliamayko @and_possum да графиня, по-моему, кончает регулярно. только мы неудовлетворенными остаемся((("


In [29]:
x_train, x_test, y_train, y_test = train_test_split(df.new_lemmas, df.label)

In [31]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [32]:
from nltk import word_tokenize

vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 300)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27940
    positive       1.00      1.00      1.00     28769

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



Как можно видеть, если оставить пунктуацию, то все метрики равны 1. 

In [33]:
len(vec.vocabulary_), len(clf.coef_[0])

(266555, 266555)

In [34]:
importances = list(zip(vec.vocabulary_, clf.coef_[0]))
importances[0]

('ну', 0.03588456855305283)

In [35]:
sorted_importances = sorted(importances, key = lambda x: -x[1])
sorted_importances[:10]

[('лучшие', 58.780667426514725),
 ('_honey_bitch_', 27.180527319648036),
 ('!', 12.59377111063971),
 ('била', 10.845916489229035),
 ('соболезнование', 9.16521786618053),
 ('норм.я', 8.032252276567048),
 ('бутылку', 7.735863469448331),
 ('@', 5.394435209535867),
 ('указа', 5.118003898803474),
 ('любимку', 4.874465978465524)]

Посмотрим, как один из наиболее значительных токенов справится с классификацией безо всякого машинного обучения:

In [36]:
cool_token = ')'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.85      0.92     32943
    positive       0.83      1.00      0.91     23766

    accuracy                           0.91     56709
   macro avg       0.91      0.92      0.91     56709
weighted avg       0.93      0.91      0.91     56709



Можно видеть, что это уже позволяет достаточно хорошо классифицировать тексты.

## Символьные n-граммы

Теперь в качестве признаком используем, например, униграммы символов:

In [37]:
vec = CountVectorizer(analyzer='char', ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


              precision    recall  f1-score   support

    negative       0.99      1.00      1.00     27855
    positive       1.00      0.99      1.00     28854

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



Таким образом, становится понятно, почему на этих данных качество классификации 1. Так или иначе, на символах классифицировать тоже можно.

Ещё одна замечательная особенность символьных признаков: токенизация и лемматизация не нужна, можно использовать такой подход для языков, у которых нет готовых анализаторов.

## Итоги

 На этом занятии мы
* познакомились с задачей бинарной классификации текстов.

* научились строить простые признаки на основе метода "мешка слов" с помощью библиотеки sklearn: CountVectorizer и TfidfVectorizer.

* использовали для классификации линейную модель логистической регрессии.

* поняли, что многое зависит от подхода к предобработки текста и от признаков, которые используются в модели.

* увидели, что в некоторых задачах важно использование каждого символа из текста, в том числе пунктуации.

На следующих занятиях мы рассмотрим более сложные модели построения признаков и классификации текстов.